In [1]:
import os
import time

import numpy as np
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision as tv
import torchvision.datasets as datasets
import art

from util import get_roc, get_shape, get_correct_examples

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {}'.format(device))

n_threads = os.cpu_count()
print('CPU threads: {}'.format(n_threads))

Device: cuda
CPU threads: 6


In [4]:
PATH = 'data'
BATCH_SIZE = 128
EPOCHS = 20

In [5]:
# Fetch dataset
transforms = tv.transforms.Compose([tv.transforms.ToTensor()])
dataset_train = datasets.MNIST(PATH, train=True, download=True, transform=transforms)
dataset_test = datasets.MNIST(PATH, train=False, download=True, transform=transforms)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
# Create Neural Network model

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, 1),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(9216, 200),
    nn.ReLU(),
    nn.Linear(200, 10),
    nn.LogSoftmax(dim=1)
)
model.to(device)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=9216, out_features=200, bias=True)
  (7): ReLU()
  (8): Linear(in_features=200, out_features=10, bias=True)
  (9): LogSoftmax(dim=1)
)

In [7]:
def train(model, loader, loss, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0.
    
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        outputs = model(x)
        l = loss(outputs, y)
        l.backward()
        optimizer.step()

        # for display
        total_loss += l.item() * batch_size
        preds = outputs.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [8]:
def validate(model, loader, loss):
    model.eval()
    total_loss = 0.
    corrects = 0.
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            outputs = model(x)
            l = loss(outputs, y)
            total_loss += l.item() * batch_size
            preds = outputs.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss=nn.NLLLoss()

In [10]:
since = time.time()
for epoch in range(EPOCHS):
    start = time.time()
    tr_loss, tr_acc = train(model, dataloader_train, loss, optimizer)
    va_loss, va_acc = validate(model, dataloader_test, loss)
    
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
time_elapsed = time.time() - since
print('Total run time: {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))

[ 1] 0m 6.3s Train Loss: 0.4175 Accuracy: 87.5767%, Test Loss: 0.1569 Accuracy: 94.9400%
[ 2] 0m 6.2s Train Loss: 0.1272 Accuracy: 96.0500%, Test Loss: 0.1009 Accuracy: 96.8800%
[ 3] 0m 6.2s Train Loss: 0.0812 Accuracy: 97.5150%, Test Loss: 0.0846 Accuracy: 97.4500%
[ 4] 0m 6.2s Train Loss: 0.0575 Accuracy: 98.2283%, Test Loss: 0.0527 Accuracy: 98.2500%
[ 5] 0m 6.2s Train Loss: 0.0433 Accuracy: 98.6867%, Test Loss: 0.0519 Accuracy: 98.3200%
[ 6] 0m 6.2s Train Loss: 0.0333 Accuracy: 98.9800%, Test Loss: 0.0465 Accuracy: 98.4500%
[ 7] 0m 6.2s Train Loss: 0.0261 Accuracy: 99.1900%, Test Loss: 0.0428 Accuracy: 98.5900%
[ 8] 0m 6.2s Train Loss: 0.0208 Accuracy: 99.3567%, Test Loss: 0.0412 Accuracy: 98.6800%
[ 9] 0m 6.2s Train Loss: 0.0163 Accuracy: 99.4900%, Test Loss: 0.0375 Accuracy: 98.8200%
[10] 0m 6.2s Train Loss: 0.0148 Accuracy: 99.5417%, Test Loss: 0.0404 Accuracy: 98.7000%
[11] 0m 6.2s Train Loss: 0.0107 Accuracy: 99.6767%, Test Loss: 0.0408 Accuracy: 98.6900%
[12] 0m 6.2s Train Lo

In [13]:
dataset_perfect = get_correct_examples(model, dataset_train, device=device)
dataloader_perfect = DataLoader(dataset_perfect, batch_size=512, shuffle=True)
_, acc = validate(model, dataloader_perfect, loss)
print('Accuracy: {:.4f}%'.format(acc*100))

Accuracy: 100.0000%
